<a href="https://colab.research.google.com/github/blabossiere/cap-comp215/blob/main/Comp_215_Project_2_Monday_April_8%2C_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import modules

%matplotlib inline

import time
from pprint import pprint

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import animation
from scipy.signal import correlate2d
import math

# Configure matplotlib's animation library to work in the browser.
matplotlib.rc('animation', html='jshtml')

In [2]:
# Qualitative colour map with value 0 set to white

tab20_mod = matplotlib.colormaps['tab20']
tab20_mod.colors = ((1,1,1,1), *tab20_mod.colors[1:])

def plot_2d_array(array, axes=None, title='', cmap=tab20_mod, **options):
    """
    Plot the 2D array as an image on the given axes  1's will be dark blue, 0's will be light blue.

    :param axes: the axes to plot on, or None to use the `plt.gca()` (current axes)
    :param options: keyword arguments passed directly to `plt.imshow()`
           see https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.imshow.html
    """
    axes = axes or plt.gca()  # If not axes are provided, draw on current axes
    axes.set_title(title)
    # Turn off axes labels and tick marks
    axes.tick_params(axis='both', which='both', bottom=False, top=False, left=False, right=False ,
                     labelbottom=False, labeltop=False, labelleft=False, labelright=False,)
    # Defaults for displaying a "matrix" with hard-pixel boundaries and (0,0) at top-left
    options = {**dict(interpolation='nearest', origin='upper'), **options}
    axes.imshow(array, cmap=cmap, **options)

In [3]:
# Animation Class

class Animation2D:
    """
      Animates any 2D model with a step() method and a draw() method, using matplotlib
      model.step() should take no parameters - just step the model forward one step.
      model.draw() should take 2 parameters, the matpltolib axes to draw on and an integer step number

      See https://www.allendowney.com/blog/2019/07/25/matplotlib-animation-in-jupyter/
          for a discussion of the pros and cons of various animation techniques in jupyter notebooks
    """

    def __init__(self, model, frames=50, steps_per_frame=1, figsize=(8, 8)):
        """
        :param model: the simulation object to animate, with step() and draw(axes, step) methods
        :param frames: number of animation frames to generate
        """
        self.model = model
        self.frames = frames
        self.steps_per_frame = steps_per_frame
        self.fig, self.ax = plt.subplots(figsize=figsize)

    def animation_step(self, step):
        """ Step the model forward and draw the plot """
        if step > 0:
            for _ in range(self.steps_per_frame):
                self.model.step()
        self.model.draw(self.ax, step=step * self.steps_per_frame)

    def show(self):
        """ return the matplotlib animation object, ready for display """
        anim = animation.FuncAnimation(self.fig, self.animation_step, frames=self.frames)
        plt.close()  # this ensures the last frame is not shown as a separate plot
        return anim

    def animate(self, interval=None):
        """ Animate the model simulation directly in the notebook display block """
        from IPython.display import clear_output
        try:
            for i in range(self.frames):
                clear_output(wait=True)  # clear the IPython display
                self.ax.clear()  # clear old image from the axes (fixes a performance issue)
                plt.figure(self.fig)  # add the figure back to pyplot ** sigh **
                self.animation_step(i)
                plt.show()  # show the current animation frame (pyplot then closes and throws away figure ** sigh **)
                if interval:
                    time.sleep(interval)
        except KeyboardInterrupt:
            pass

In [4]:
['Red', 'Green', 'Blue']
[0.5, 0, 1]

class Landscape:

  # Define names for the 2 possible cell states of landscape.
  BARREN = 0
  GRASS = 1

  # Define a colour map that maps each cell state to an intuitive colour.
  cmap = [(0.737, 0.56, 0.56), (0.87,0.722,0.529)]
  cmap[BARREN] = (0.87,0.722,0.529)  # brown
  cmap[GRASS] = (0.1333,0.545,0.1333)  # green
  landscape_colour_map = matplotlib.colors.ListedColormap(cmap)

  def __init__(self, n=10, q=0.5, r=0.3):
    """
    n: the column and row size of the array
    q: the probability that the intial landscape is filled with grass.
    r: the probability that the barren cells regrow into grass-filled cells.
    """

    self.n = n
    self.q = q
    self.r = r

  #def make_og_landscape(self):
    #self.state = np.zeros(shape = (self.n, self.n), dtype=np.uint8)
    self.state = np.random.choice([self.GRASS, self.BARREN], (self.n, self.n), p=[self.q, 1 - self.q])

  def step(self):
    for y in range(self.n):
      for x in range(self.n):
        if self.state[y][x] == 0:
          new_number = np.random.choice([self.GRASS, self.BARREN], (1, 1), p=[self.r, 1 - self.r])
          self.state[y][x] = new_number

  def num_occupied(self):
        """ return the number of cells occupied by forest """
        grass_count = 0
        for row in range(self.n):
          for row_index in range(self.n):
            if self.state[row][row_index] == 1:
              grass_count = grass_count + 1
        return grass_count  # TODO: count the number of forested cells.


  def pct_occupied(self):
    """ return the proportion of cells occupied by forest """
    return self.num_occupied() / self.state.size


  def draw(self, axes=None, step=''):
    """Draws the CA cells using the forest colour map so values are coloured intuitively."""
    axes = axes or plt.gca()
    title = f'Time:{step} Occupied: {round(self.pct_occupied() * 100, 2)}%'
    plot_2d_array(self.state, axes=axes, title=title,
                  cmap=self.landscape_colour_map, vmin=0, vmax=len(self.landscape_colour_map.colors))

# print(cmap[BARREN])
# print(landscape_colour_map)

grassland1 = Landscape(n=10, q=0.5, r=0.3)
#grassland1.make_og_landscape()
grassland1.state
# grassland1.step()
grassland1.state

array([[1, 1, 1, 1, 1, 1, 0, 0, 1, 0],
       [1, 1, 0, 0, 1, 1, 0, 0, 1, 1],
       [1, 1, 1, 1, 1, 0, 1, 0, 0, 1],
       [1, 0, 0, 1, 0, 1, 1, 1, 1, 0],
       [0, 1, 0, 0, 0, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 1],
       [0, 0, 0, 0, 1, 0, 0, 1, 0, 1],
       [1, 1, 0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 1, 0, 1, 1, 1, 1]])

In [13]:
class Deermap:

  # Define names for the 11 possible cell states
  NO_DEER = 0
  DEER = 1



  # Define a colour map that maps each deer state to an colour, with a gradient of blue representing the health status of each deer.
  cmap2 = [(1, 1, 1), (0.7, 0.7, 1), (0.6, 0.6, 1), (0.5, 0.5, 1), (0.4, 0.4, 1), (0.3, 0.3, 1), (0.2, 0.2, 0.9), (0.1, 0.1, 1), (0, 0, 0.9), (0, 0, 0.7), (0, 0, 0.5)]
  deer_colour_map = matplotlib.colors.ListedColormap(cmap2)

  def __init__(self, n=10, d=0.3):
    """
    n: the column and row size of the array
    d: the initial proportion of the landscape occupied by deer.
    """

    self.n = n
    self.d = d

    #self.state = np.zeros(shape = (self.n, self.n), dtype=np.uint8)
    self.state = np.random.choice([self.DEER, self.NO_DEER], (self.n, self.n), p=[self.d, 1 - self.d])

  #def make_og_deer_map(self):
    for y in range(self.n):
      for x in range(self.n):
        if self.state[y][x] == 1:
          new_number = np.random.choice(range(1, 11))
          self.state[y][x] = new_number

  def num_occupied(self):
    """ return the number of cells occupied by forest """
    deer_count = 0
    for row in range(self.n):
      for row_index in range(self.n):
        if self.state[row][row_index] > 0:
          deer_count = deer_count + 1
    return deer_count  # TODO: count the number of forested cells.


  def pct_occupied(self):
    """ return the proportion of cells occupied by forest """
    return self.num_occupied() / self.state.size


  def draw(self, axes=None, step=''):
    """Draws the CA cells using the forest colour map so values are coloured intuitively."""
    axes = axes or plt.gca()
    title = f'Time:{step} Occupied: {round(self.pct_occupied() * 100, 2)}%'
    plot_2d_array(self.state, axes=axes, title=title,
                  cmap=self.deer_colour_map, vmin=0, vmax=len(self.deer_colour_map.colors))

deer_map1 = Deermap(n=10, d=0.3)
#deer_map1.make_og_deer_map()
deer_map1.state

array([[ 0,  0,  0,  8,  0,  0,  7,  0,  5,  0],
       [ 0,  0,  0,  6,  2,  0,  0,  0,  0,  0],
       [ 0,  2,  0,  0,  0,  0,  0,  0,  0,  8],
       [ 0,  8,  0,  0,  3,  0,  2,  0,  0,  0],
       [ 0,  0,  9,  0,  3,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  9,  6,  0,  6],
       [ 0,  0,  0,  0,  6,  0,  0,  0,  0,  0],
       [ 4,  0,  0,  5,  7,  0,  8,  0,  9,  0],
       [ 0,  7,  9,  3,  0, 10,  0,  0,  1,  0],
       [ 4,  0,  0,  0,  0,  0,  0, 10,  2,  0]])

In [6]:
chris = Deermap(n=10, d=0.3)
print(chris.state)
#steve = ForestFire(20, p=0.5, f=0.2)
animator = Animation2D(chris, frames=1, steps_per_frame=1, figsize=(8, 8))
animator.show()
#animator.animate()

[[ 0  0 10  0 10  0  0  7  8  0]
 [ 0  3 10  5  0  3  0  0  0 10]
 [ 6  0  0  0  0  7  0  8  0  0]
 [ 0  1  0  0  0  0  0  0  0  0]
 [ 0  1  0  0  0  0  0  0  4  0]
 [ 0  9  1  0  0  3  0  0  9  0]
 [ 0  0  0  0  0 10  0  0  0  0]
 [ 0  0  0  0  4  2  7  0  3 10]
 [ 0  3  6  0  5  5  0  0  0  2]
 [ 0  0  3  0  0  0  0  0  0  0]]


In [8]:
dan = Landscape(n=10, q=0.5, r=0.3)
#steve = ForestFire(20, p=0.5, f=0.2)
animator = Animation2D(dan, frames=20, steps_per_frame=1, figsize=(8, 8))
animator.show()
#animator.animate()

<ipython-input-4-8dd33d83cc2f>:36: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  self.state[y][x] = new_number


In [27]:
# deermap
# grassmap

# deer_on_grass = deermap!=EMPTY & grassmap!=EMPTY
# deermap[deer_on_grass] += 1
# grass[deer_on_grass] = EMPTY

class Deer_and_Grass_Map:

  def __init__(self, n=10, q=0.5, r=0.3, d=0.3):
    self.n = n
    self.q = q
    self.r = r
    self.d = d

    self.deermap = Deermap(self.n, self.d)
    self.grassmap = Landscape(self.n, self.q, self.r)

  def step():
    a


super_map = Deer_and_Grass_Map(n=10, q=0.5, r=0.3, d=0.3)
super_map.deermap.state
super_map.grassmap.state

array([[0, 0, 0, 1, 1, 0, 1, 1, 0, 1],
       [1, 0, 1, 0, 1, 0, 1, 1, 1, 1],
       [0, 0, 1, 1, 1, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 1, 0, 0, 1, 0, 1],
       [0, 0, 0, 1, 1, 0, 1, 1, 1, 1],
       [0, 1, 1, 0, 1, 1, 1, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 1, 0, 1],
       [1, 0, 0, 1, 1, 1, 1, 0, 1, 1],
       [1, 1, 1, 1, 0, 1, 0, 0, 1, 0],
       [0, 0, 1, 0, 0, 1, 0, 1, 1, 1]])